### 05/06/2019 To-do list:
1. Calibration curve analog for regression (edited) 
2. Indicate how many points are below/above each parity line
3. Plot multiple cases side by side on the same graph (e.g. test set with 50 pts & 1500 pts)
4. Manipulating dropout

### 06/14/2019 To-do list:
1. If process gets stuck on SDT transform (tqdm) step:
   a. Cache

This document demonstrates the making, training, saving, loading, and usage of a sklearn-compliant CGCNN model.

In [1]:
%load_ext ipycache

/pylon5/ch5fq5p/zulissi/miniconda3/envs/cgcnn/lib/python3.6/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated since IPython 4.0. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/pylon5/ch5fq5p/zulissi/miniconda3/envs/cgcnn/lib/python3.6/site-packages/ipycache.py:17: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  from IPython.utils.traitlets import Unicode


In [2]:
import os
import sys
#Comment/add these
sys.path.insert(0,'../')
sys.path.insert(0,'/home/zulissi/software/adamwr/')

import numpy as np
import cgcnn

import time

#Select which GPU to use if necessary
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0


## Load the dataset as mongo docs

In [3]:
import random
import pickle

starttime = time.clock()
#Load a selection of documents
docs = pickle.load(open('/pylon5/ch5fq5p/zulissi/CO_docs.pkl','rb'))
random.seed(42)
random.shuffle(docs)
docs = [doc for doc in docs if -3<doc['energy']<1.0]
docs = docs[:6000]

endtime = time.clock()
print('This operation took', endtime - starttime, 's.')

This operation took 14.900000000000002 s.


## Get the size of the features from the data transformer, to be used in setting up the net model

In [ ]:
# %%cache SDT_list.pkl SDT_list
from torch.utils.data import Dataset, DataLoader
import mongo
from cgcnn.data import StructureData, ListDataset, StructureDataTransformer
import numpy as np
import tqdm
from sklearn.preprocessing import StandardScaler

SDT = StructureDataTransformer(atom_init_loc='../atom_init.json',
                              max_num_nbr=12,
                               step=0.2,
                              radius=1,
                              use_tag=False,
                              use_fixed_info=False,
                              use_distance=True)

SDT_out = SDT.transform(docs)

structures = SDT_out[0]

# Settings necessary to build the model (since they are size of vectors as inputs)
orig_atom_fea_len = structures[0].shape[-1]
nbr_fea_len = structures[1].shape[-1]

import multiprocess as mp
from sklearn.model_selection import ShuffleSplit

SDT_out = SDT.transform(docs)

with mp.Pool(4) as pool:
    SDT_list = list(tqdm.tqdm(pool.imap(lambda x: SDT_out[x],range(len(SDT_out)),chunksize=40),total=len(SDT_out)))


 20%|██        | 1201/6000 [07:26<24:25,  3.27it/s]  

In [ ]:
starttime = time.clock()
with open('distance_all_docs.pkl','wb') as fhandle:
    pickle.dump(SDT_list,fhandle)

endtime = time.clock()
print('This step took', endtime - starttime, 's to complete.')

## CGCNN model with skorch to make it sklearn compliant

In [ ]:
from torch.optim import Adam, SGD
from sklearn.model_selection import ShuffleSplit
from skorch.callbacks import Checkpoint, LoadInitState #needs skorch 0.4.0, conda-forge version at 0.3.0 doesn't cut it
from cgcnn.data import collate_pool
from skorch import NeuralNetRegressor
from cgcnn.model_no_dropout import CrystalGraphConvNet
import torch
from cgcnn.data import MergeDataset
import skorch.callbacks.base

cuda = torch.cuda.is_available()
if cuda:
    device = torch.device("cuda")
else:
    device='cpu'

startime = time.clock()

#Make a checkpoint to save parameters every time there is a new best for validation lost
cp = Checkpoint(monitor='valid_loss_best',fn_prefix='valid_best_')

#Callback to load the checkpoint with the best validation loss at the end of training
class train_end_load_best_valid_loss(skorch.callbacks.base.Callback):
    def on_train_end(self, net, X, y):
        net.load_params('valid_best_params.pt')
        
load_best_valid_loss = train_end_load_best_valid_loss()

endtime = time.clock()
print('This step takes', endtime - startime, 's to complete.')

\color{red}{This seems to be a time consuming step.}

## Example converting all the documents up front

In [ ]:
starttime = time.clock()
#Make the target list
target_list = np.array([doc['energy'] for doc in docs]).reshape(-1,1)

endtime = time.clock()
print('This step takes', endtime - startime, 's to complete.')

## Shuffle and Split

In [ ]:
from sklearn.model_selection import train_test_split

starttime = time.clock()
SDT_training, SDT_test, target_training, target_test = train_test_split(SDT_list, target_list, test_size=0.2)

endtime = time.clock()
print('This step takes', endtime - startime, 's to complete.')


# Fit the model

In [ ]:
from skorch.dataset import CVSplit
from skorch.callbacks.lr_scheduler import WarmRestartLR, LRScheduler

train_test_splitter = ShuffleSplit(test_size=0.25) # , random_state=42)
LR_schedule = LRScheduler('MultiStepLR',milestones=[100],gamma=0.1)

net = NeuralNetRegressor(
    CrystalGraphConvNet,
    module__orig_atom_fea_len = orig_atom_fea_len,
    module__nbr_fea_len = nbr_fea_len,
    # module__dropout = 0.2,
    batch_size=214,
    module__classification=False,
    lr=0.0056,
    max_epochs=188, # 292
    module__atom_fea_len=46,
    module__h_fea_len=83,
    module__n_conv=8,
    module__n_h=4,
    optimizer=Adam,
    iterator_train__pin_memory=True,
    iterator_train__num_workers=0,
    iterator_train__collate_fn = collate_pool,
    iterator_train__shuffle=True,
    iterator_valid__pin_memory=True,
    iterator_valid__num_workers=0,
    iterator_valid__collate_fn = collate_pool,
    device=device,
    criterion=torch.nn.MSELoss,
#   criterion=torch.nn.L1Loss,
    dataset=MergeDataset,
    train_split = CVSplit(cv=train_test_splitter),
    callbacks=[cp, load_best_valid_loss, LR_schedule]
)

In [ ]:
#### Change This!!!

import pandas as pd

import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def plot(SDT_training, SDT_test, target_training, target_test, train_indices, valid_indices, net):
    # train_indices, valid_indices = next(train_test_splitter.split(SDT_training))
    # lentotal = len(SDT_training)
    # train_indices = list(range(round(3/4 * lentotal)))
    # valid_indices = list(range(round(3/4 * lentotal), lentotal))

    training_data = {'actual_value':np.array(target_training.reshape(-1))[train_indices],
                     'predicted_value':net.predict(SDT_training)[train_indices].reshape(-1)}
    test_data = {'actual_value':np.array(target_test).reshape(-1),
                 'predicted_value':net.predict(SDT_test).reshape(-1)}
    validation_data = {'actual_value':np.array(target_training.reshape(-1))[valid_indices],
                       'predicted_value':net.predict(SDT_training)[valid_indices].reshape(-1)}

    df_training = pd.DataFrame(training_data)
    df_validation = pd.DataFrame(validation_data)
    df_test = pd.DataFrame(test_data)
    
    f, ax = plt.subplots(figsize=(8,8))
    ax.scatter(df_training['actual_value'], df_training['predicted_value'], color='orange', 
               marker='o', alpha=0.5, label='train\nMAE=%0.2f, RMSE=%0.2f, R$^2$=%0.2f'\
                %(mean_absolute_error(df_training['actual_value'], df_training['predicted_value']), 
                  np.sqrt(mean_squared_error(df_training['actual_value'], df_training['predicted_value'])),
                  r2_score(df_training['actual_value'], df_training['predicted_value'])))

    ax.scatter(df_validation['actual_value'], df_validation['predicted_value'], color='blue', 
               marker='o', alpha=0.5, label='valid\nMAE=%0.2f, RMSE=%0.2f, R$^2$=%0.2f'\
                %(mean_absolute_error(df_validation['actual_value'], df_validation['predicted_value']), 
                  np.sqrt(mean_squared_error(df_validation['actual_value'], df_validation['predicted_value'])),
                  r2_score(df_validation['actual_value'], df_validation['predicted_value'])))

    ax.scatter(df_test['actual_value'], df_test['predicted_value'], color='green', 
               marker='o', alpha=0.5, label='test\nMAE=%0.2f, RMSE=%0.2f, R$^2$=%0.2f'\
                %(mean_absolute_error(df_test['actual_value'], df_test['predicted_value']), 
                  np.sqrt(mean_squared_error(df_test['actual_value'], df_test['predicted_value'])),
                  r2_score(df_test['actual_value'], df_test['predicted_value'])))


    ax.plot([min(df_training['actual_value']), max(df_training['actual_value'])], 
            [min(df_training['actual_value']), max(df_training['actual_value'])], 'k--')

    # format graph
    ax.tick_params(labelsize=14)
    ax.set_xlabel('DFT E (eV)', fontsize=14)
    ax.set_ylabel('CGCNN predicted E (eV)', fontsize=14)
    ax.set_title('Multi-element ', fontsize=14) 
    ax.legend(fontsize=12)

    plt.show()
    
    return df_training, df_validation, df_test

In [ ]:
from sklearn.model_selection import train_test_split

TrainingData = []
ValidationData = []
TestData = []

iters = 7

starttime = time.clock()

train_test_splitter = ShuffleSplit(test_size=0.25, random_state=42)
train_indices, valid_indices = next(train_test_splitter.split(SDT_training))

net.initialize()
net.fit(SDT_training, target_training)

for i in range(iters):
    # net()
    # net.initialize()
    train_test_splitter = ShuffleSplit(test_size=0.25, random_state=42)
    train_indices, valid_indices = next(train_test_splitter.split(SDT_training))
    
    print("train_indices:", train_indices)
    print("valid_indices:", valid_indices)
    
    with open('no-dropout_log.txt', 'a') as logfile:
        logfile.write("Iter: %s" % (i,))
        logfile.write("train_indices: %s" % (train_indices,))
        logfile.write("train_indices: %s\n" % (train_indices,))

    # net.fit(SDT_training, target_training)    
    dftraining, dfvalidation, dftest = plot(SDT_training,
                                            SDT_test,
                                            target_training,
                                            target_test,
                                            train_indices,
                                            valid_indices, net)
    
    TrainingData.append(dftraining)
    ValidationData.append(dfvalidation)
    TestData.append(dftest)
    
TrainingData = pd.concat(TrainingData, axis=1)
ValidationData = pd.concat(ValidationData, axis=1)
TestData = pd.concat(TestData, axis=1)

endtime = time.clock()
print("Calculating the same points {} times takes {} s.".format(iters, endtime-starttime))

In [ ]:
"""
# The d20 suffix means a droupout of 20% is applied
TrainingData.to_pickle('TrData_7iters_d20.pkl')
ValidationData.to_pickle('VlData_7iters_d20.pkl')
TestData.to_pickle('TsData_7iters_d20.pkl')
"""

In [ ]:
"""
# The d30 suffix means a droupout of 30% is applied
TrainingData.to_pickle('TrData_7iters_d30.pkl')
ValidationData.to_pickle('VlData_7iters_d30.pkl')
TestData.to_pickle('TsData_7iters_d30.pkl')
"""

In [ ]:
TrainingData.to_pickle('TrData_7iters_vanilla_081319.pkl')
ValidationData.to_pickle('VlData_7iters_vanilla_081319.pkl')
TestData.to_pickle('TsData_7iters_vanilla_081319.pkl')
